In [ ]:
%pip install pygeohash
%pip install haversine

Python interpreter will be restarted.
Collecting pygeohash
 Using cached pygeohash-1.2.0-py2.py3-none-any.whl
Installing collected packages: pygeohash
Successfully installed pygeohash-1.2.0
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting haversine
 Using cached haversine-2.7.0-py2.py3-none-any.whl (6.9 kB)
Installing collected packages: haversine
Successfully installed haversine-2.7.0
Python interpreter will be restarted.

In [ ]:
import sys
import math
import pandas as pd
import os
# from pyais import decode, IterMessages
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
import numpy as np
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import itertools
from itertools import *
from haversine import haversine, Unit
import sys
from pyspark.sql import Window
import pyspark.sql.functions as F
import pygeohash
import re

In [ ]:
def Ship_position_reports(df):
    """
    This function retuns the dataframe with dynamic ais messages
    
    """
 
    dynamic_col = ['mmsi','msg_type','status','lon','lat','heading','course','speed','timestamp']
    #select the static messages
    msg_type_filter = col('msg_type').isin([1,2,3,18,27])
    AIS_Dynamic_MSG = df.select(dynamic_col).filter(msg_type_filter)
    AIS_Dynamic_MSG = AIS_Dynamic_MSG.filter(valid_course & valid_heading & valid_lon & valid_lat)
    
    Status_code = {0.0 : "Under way using its engine",
                1.0 : "Anchored",
                2.0: "Not under command",
                3.0 : "Has restricted maneuverability",
                4.0: "Ship draught is limiting its movement",
                5.0 : "Moored",
                6.0: "Aground",
                7.0: "Engaged in fishing",
                8.0: "Under way sailing",
                9.0: "carrying dangerous goods",
                10.0 : "carrying dangerous goods",
                11.0: "Power-driven vessel towing astern",
                12.0: "Power-driven vessel pushing ahead",
                13.0: "(Reserved for future use)",
                14.0: "Emergency",
                15.0: "Default",
                99: "Unknow"
              }

    mapping_expr = create_map([lit(x) for x in chain(*Status_code.items())])
    #fill missing value
    AIS_Dynamic_MSG = AIS_Dynamic_MSG.fillna(99.0,['status'])
    AIS_Dynamic_MSG = AIS_Dynamic_MSG.withColumn('status',mapping_expr[AIS_Dynamic_MSG['status']])
    

    return AIS_Dynamic_MSG

In [4]:
def static_report_vessel(df):
    """
    This function retuns the dataframe with static ais messages
    
    """
    def Vessel_type_mapping(x):
                if (x >=20 and x<30):

                        return "Wing In Ground"
                elif (x==51):

                        return "search and rescue"
                elif  (x==31):

                        return "Fishing Vessel"
                elif(x==31 or x ==32 or x ==33):

                        return 'Tug'
                elif ((x>33 and x <= 35) or (x>=50 and x <= 59 )):

                        return "Fishing Vessel"
                elif(x == 36):
        
                        return 'Special Craft'
                elif (x == 37):

                        return 'Pleasure Craft'
                elif (x == 36):
                        return 'Sailing Vessel'
                
                elif (x ==38 or x == 39):
        
                        return 'Reserved'    
                
                elif (x >=40 and x<50):
                
                        return 'High-Speed Craft' 
                
                elif (x>=60 and x<70):
                
                        return 'Passenger Ship'
                        
                elif (x>=70 and x< 80):
                        
                        return 'Cargo'
                
                elif (x>=80 and x< 90):
        
        
                        return 'Tanker'
        
                elif (x>=90 and x< 100):
                        
                        return 'Other'
                else : 
                        return 'Unknown'
                
            

    # select the dynamics messages
    msg_type_filter = col('msg_type').isin([5,19,24,18])
    AIS_Static_MSG = df.filter(msg_type_filter) 
    
    #calculate ship_length & ship_width
    AIS_Static_MSG = AIS_Static_MSG.withColumn("ship_area",col('to_bow')+col('to_stern')*( col('to_starboard') + col('to_port')))
    
    #calculte the lengh of ship
    AIS_Static_MSG = AIS_Static_MSG.withColumn("ship_length",col('to_bow')+col('to_stern'))
     
    #calculte the width of ship
    AIS_Static_MSG = AIS_Static_MSG.withColumn("ship_width",col('to_starboard') + col('to_port'))
    AIS_Static_MSG = AIS_Static_MSG.select('mmsi','imo','callsign','shipname','ship_type','ship_area','ship_length','ship_width','draught','destination','eta')
    
    
    
    #regist dict mapping udf function
    Vessel_type_map = udf(lambda x: Vessel_type_mapping(x), StringType())
    #use 10086 to represent the missing ship type
    AIS_Static_MSG = AIS_Static_MSG.fillna(10086,['ship_type']) #replace missing value with same data type
    #dict mapping
    AIS_Static_MSG = AIS_Static_MSG.withColumn('ship_type',Vessel_type_map(col("ship_type")))
    
    #fill missing value
    for index, column in enumerate(AIS_Static_MSG.columns):
        if column == 'destination' or column == 'callsign':
            AIS_Static_MSG = AIS_Static_MSG.na.fill("Unknown")
            continue
        AIS_Static_MSG = AIS_Static_MSG.fillna(0,column)
    #drop duplicates
    
    AIS_Static_MSG = AIS_Static_MSG.dropDuplicates(['mmsi'])

    return AIS_Static_MSG

In [5]:
def GeoHash(df,grid_level):
    """
    This function retun the dataframe with geohash gird
    
    """
    # map geo location into geohash
    def convertCase(x,y):
        location_gird = pygeohash.encode(x,y,grid_level)
#         location  = pygeohash.decode(location_gird)
        #return location grid informat of latitude and longitude
        return location_gird 
    # Converting function to UDF
    convertUDF = udf(convertCase, StringType())

    #Geo hash mapping
    Ship_grid = df.withColumn('grid', convertUDF('lat', 'lon'))
    
    return Ship_grid

def TimeWindow(df,seconds):
    """
    This function retun the dataframe group by time widnow
    seconds: time to scan the geohash grid
    
    """
    # create time window for every 5 min
    seconds_window = F.from_unixtime(F.unix_timestamp('time') - F.unix_timestamp('time') % seconds)
    Ship_grid = df.withColumn('time',col('timestamp').cast(TimestampType()))
    Ship_grid_time = Ship_grid.withColumn('5_minutes_window', seconds_window)


    #only keep the one ais msg for in every 5 min
    Ship_grid_time = Ship_grid_time.dropDuplicates(['5_minutes_window','mmsi'])

    #calulate avg speed in every 5 min for each ships
    w = Window.partitionBy('mmsi','5_minutes_window')
    Ship_grid_time_avg_speed = Ship_grid_time.withColumn('avg_speed', F.mean('speed').over(w))
    return Ship_grid_time

In [6]:
def STS_meeting(dynamic_df,static_df):
    """
    STS:
        2 or more different vessels meet in same grid （close to each other） and last more than 30 minutes
    """
        

    mid_point = udf(get_geo_mid,  ArrayType(FloatType()))

    #   Get the duplicated ship info in per time window
    Ship_grid_distinct = dynamic_df.dropDuplicates(['5_minutes_window','mmsi','grid']).groupBy('5_minutes_window','grid','mmsi').count()

    #  meeting for every 5 minute
    Ship_grid_distinct.createOrReplaceTempView("E")
    Ship_grid_distinct.createOrReplaceTempView("D")
  
    current = spark.sql("select * from E e, D d where e.5_minutes_window == d.5_minutes_window and e.grid == d.grid and e.mmsi != d.mmsi ").\
    select('e.5_minutes_window','e.grid','e.mmsi',col('d.mmsi').alias('mmsi2'))
#            col('e.lat').alias('lat1'),col('e.lon').alias('lon1'),col('d.lat').alias('lat2'),col('d.lon').alias('lon2'))

    # check the meeting last more than 5 min
    current.createOrReplaceTempView("current")
    current.createOrReplaceTempView("future")
    future = spark.sql("select * from current c, future f where f.5_minutes_window >c.5_minutes_window  and c.mmsi2 == f.mmsi2 and c.mmsi == f.mmsi").\
    select(col('c.5_minutes_window').alias('start'),  col('f.5_minutes_window').alias('5_min_later'), col('c.grid').alias('area'),'c.mmsi','c.mmsi2')
#            'c.lat1','c.lon1','c.lat2','c.lon2')
    # calculate meeting duration
    future = future.withColumn('duration',F.unix_timestamp('5_min_later') - F.unix_timestamp('start'))
    
    #meeting duration more than 0.5 hour

    # only keep the time when meeting begin and end
    w = Window.partitionBy('c.mmsi','c.mmsi2')
    #single sts meeting
    sts_meeting = future.withColumn('start_time', F.min('start').over(w))\
        .withColumn('end_time', F.max('5_min_later').over(w))\
        .where(( F.col('5_min_later') == F.col('end_time') ) & (F.col('start_time') == F.col('start')))
    #cast the mmsi into array to union with meeting list
     
    #multi sts meeting
    #cast the mmsi into array to union with meeting list
    
    
    #transform mmsi in format of array to joint with other mmsi array e.g ‘【123241413】 【12312414124,3423425,34234234】’
    sts_meeting = sts_meeting.withColumn('mmsi_', array(col('mmsi')))
    sts_meeting = sts_meeting.withColumn('hour', hour('start_time'))



    #only keep the qualified STSmeeting
    duration = col('duration')>=1800
    #Get the table of STS meeting and Multi STSmeeting 
    meeting_list =     sts_meeting.filter(col('duration')>=1800).groupBy('mmsi','mmsi_','start', 'hour','area')\
            .agg(collect_list('mmsi2').alias('meeting_with'))\
            .withColumn('meeting', array_union(col("mmsi_"),col('meeting_with')))\
            .withColumn('meeting', array_sort(col('meeting'))).withColumnRenamed('area2','area')
    #generate unique meeting id for each STS meeting
    meeting_ID = meeting_list.dropDuplicates(['meeting']).withColumn("meeting_id", monotonically_increasing_id()).select('meeting','meeting_id')




    #assin meeting id to every meeting 
    meeting_list_with_id = meeting_list.join(meeting_ID,['meeting'])
    meeting_list_with_id = meeting_list_with_id.select('meeting','meeting_id',col('hour').alias('hour2'),col('mmsi').alias('mmsi2'),'area','start','meeting_with')
    

    #augment info (dynamic & stastic) for each sts meeing
    dynamic_df = dynamic_df.withColumn('hour',hour('5_minutes_window'))
    
    Final_sts_meeting = meeting_list_with_id.join(dynamic_df, (dynamic_df.mmsi == meeting_list_with_id.mmsi2)  & (meeting_list_with_id.start == col('5_minutes_window')) & (dynamic_df.grid == meeting_list_with_id.area) & (dynamic_df.hour == meeting_list_with_id.hour2) ,'inner')\
                               .join(static_df,['mmsi'],'left')\
                               .drop('mmsi2')
    
    

#     fill missing value
    for index, column in enumerate(Final_sts_meeting.columns):
            # fill null with "Unknown" for static info
            if column in (['callsign','shipname','ship_type','destination','eta']):
                Final_sts_meeting = Final_sts_meeting.na.fill("Unknown")
                continue
            Final_sts_meeting = Final_sts_meeting.fillna(0,column)
    return Final_sts_meeting

In [7]:
def output_file(df):
    """  select the cols that are useful for visualization  """
    
    
    #crate unique index for visualization on map e.g meetingid-mssi
    Final_sts_meeting = df.withColumn('mmsi',col('mmsi').cast(StringType()))
    Final_sts_meeting = Final_sts_meeting.withColumn('meeting_id',col('meeting_id').cast(StringType()))
    Final_sts_meeting = Final_sts_meeting.withColumn('name',concat(col("meeting_id"),lit("-"),col("mmsi")))
    Final_sts_meeting = Final_sts_meeting.withColumn('min',minute('start'))


    
    #select the row for visualization
    output =  Final_sts_meeting.select(col('name'),col('shipname').alias('ship_name'),col("lat").alias('latitude'),col("lon").alias('longitude'),'speed','heading','course','status','ship_type','draught','destination','eta',col('ship_length').alias('length'),col('ship_width').alias('width'),'imo','meeting','meeting_id',col('start').alias('meeting_time'),col('hour'),'min','mmsi','meeting_with')
                                       

    return output

In [ ]:
filelist = dbutils.fs.ls('/mnt/lsde/group12/input/')
out_put_dir = '/mnt/lsde/group12/output-minute-final/'
# dbutils.fs.rm(out_put_dir,recurse = True)
counter = []
#process data day by day 
for file in filelist[:]:
    
    #get the date and generate output file path
    date= re.sub('-0','-',file[0].split('/')[-2])
    out_out_file_dir = out_put_dir+date
    
    df = spark.read.option("header", "false").option("delimiter", ",").parquet(file[0])
    #get the vessel static data
    AIS_Static_MSG_df = static_report_vessel(df)
    #get the vessel dynamic data
    Ship_AIS_Dynamic_df = Ship_position_reports(df)
    #convert location in to grid
    Ship_grid = GeoHash(Ship_AIS_Dynamic_df,6)
    #group by data with time window 
    Ship_grid_time =TimeWindow(Ship_grid,300)
    #caculate sts meeting
    Final_sts_meeting = STS_meeting(Ship_grid_time,AIS_Static_MSG_df)
    #output file
    output=  output_file(Final_sts_meeting)



    #save file in format of json
    output.coalesce(1).write.partitionBy("meeting_time").mode("Overwrite").format('json').save(out_out_file_dir)
    file = dbutils.fs.ls('/mnt/lsde/group12/output-minute-final/'+date)
    path = '/mnt/lsde/group12/output-minute-final/'+date
    #delete redundant file output by Spark
    for fl in file[:]:
            #check the whether it is  json fil
        if 'hour'in fl[0].split('/')[-2]:
            for minute_ in dbutils.fs.ls(fl[0]):
                for i in  dbutils.fs.ls(minute_[0]):
                    #check  whether it is json file
                    if '.json'  not in i[0].split('/')[-1]:
                        dbutils.fs.rm(i[0],recurse = True)
    
                    else:

                   #change the name of raw json file
                      hour_ = re.findall(r"\d+\.?\d*",fl[0].split('/')[-2])
                   #delete the original directory
                      min_ = re.findall(r"\d+\.?\d*",minute_[0].split('/')[-2])
                      dbutils.fs.mv(i[0], 'dbfs:'+out_put_dir+date+'/hour='+str(hour_[0])+'/ships-'+date+'-'+str(hour_[0])+'-'+min_[0].zfill(2)+'.json', True)
                dbutils.fs.rm(minute_[0])
    print(date+' is'+' ready')
                        